# Get Data from Airtable

This gets the data from the two tables and formats it into dataframes. It's probably not the cleanest thing in the world, but hopefully it'll do for now.

In [63]:
import ast
import pandas as pd
import requests
import textwrap
import simplejson
from datetime import datetime

In [81]:
LINE_LENGTH = 15
TESTING = True

jsonp_body = "// This is crusty, but it seems to work.\n// Generated at {}".format(str(datetime.now()))

In [73]:
def mergeD(ugly_data):
    """Put the ID inside the dictionary that becomes the row."""
    f = ugly_data["fields"]
    i = {"id": ugly_data["id"]}
    f.update(i)
    return f

if TESTING:
    t = mergeD({'createdTime': '2018-05-25T07:45:33.000Z', 'fields': {'Other_people_involved': ['rece9qyNSfMqAhZAL'], 'Owner': ['recgcDkgKvrBLH1k0'], 'LongName': 'A Plane of Thrones Paper', 'Name': 'potPaper', 'percentComplete': 100, 'precursors': ['recflCm8BzuGQbmW0', 'recSIIp1grVggDGQo'], 'Type': 'Publication'}, 'id': 'rec0ArDcPS4hLrmJq'})
    print(t)

In [74]:
def get_table(table_name):
    """Get the table from the OATMEAL airtable database and return it as a DataFrame."""
    auth_headers = {'Authorization': 'Bearer keyYXkjYFw61SeWDk'}
    table_url = 'https://api.airtable.com/v0/appuaXpFiadmP89sq/' + table_name
    r = requests.get(table_url, headers=auth_headers)
    d = eval(ast.literal_eval(str(r.content)[1:])) # this is almost certainly not the right way to do this!
    
    rows = [mergeD(x) for x in d["records"]]
    data_frame = pd.DataFrame(rows)
    return data_frame

if TESTING:
    t = get_table("Techs")
    print(t)

In [87]:
def pull_out_embed_url(embed_code):
    """Pull the embed url out of the iframe link."""
    if type(embed_code) is not float:
        for thing in embed_code.split(" "):
            if "src=" in thing:
                thing = thing.split("\"")
                for part in thing:
                    if "https://" in part:
                        return part.split("?")[0]
    else:
        return "https://www.youtube.com/watch?v=dQw4w9WgXcQ"

if TESTING:
    print( pull_out_embed_url(project_data.loc["recgrgZIuBEI9namY"].deck_embed) )

None


In [ ]:
def wrap_long_name(row):
    """Insert line breaks to make names fit in the node."""
    try:
        ln = "\n".join( textwrap.wrap(row["LongName"], LINE_LENGTH))
    except Exception as e:
        ln = row["LongName"]
    return ln

In [ ]:
def make_node_description(row, long_name_broken):
    node_description_pattern = "{name} [label=\"{longName}\"];"
    return node_description_pattern.format(name=row["Name"], longName=long_name_broken)

In [88]:
project_data = get_table("Techs")
project_data.set_index("id", inplace=True)
project_data.deck_embed = project_data.deck_embed.apply(pull_out_embed_url)
project_data.sample(2)

,LongName,Name,Other_people_involved,Owner,Type,deck_embed,percentComplete,precursors
id,,,,,,,,
recF1PTEM9uexmaTm,Timesheet Data,timeData,NaN,NaN,Capability,https://www.youtube.com/watch?v=dQw4w9WgXcQ,0,NaN
reczTE03dicIgyCUJ,Buildings That Are Right All The Time,BTARATA,NaN,NaN,Capability,https://www.youtube.com/watch?v=dQw4w9WgXcQ,0,[recKHYcTAaR6Vkn3s]


In [37]:
edge_pattern = "{from_node} -> {to_node};"

node_descriptions = []
edges = []
for i, row in project_data.iterrows():
    ln = wrap_long_name(row)
    n = make_node_description(row, ln)
    node_descriptions.append(n)
    
    if type(row["precursors"]) is not float: # nothing reads as NaN, therefore float
        precursors = row["precursors"]#.split(",")
        for precursor in precursors:
            e = edge_pattern.format(from_node=project_data.loc[precursor].Name,
                                    to_node=row["Name"])
            edges.append(e)
    

In [39]:
g = """digraph G {{
  rankdir  = "LR";
  label    = "Tech Tree of OATMEAL Projects";
  labelloc = "t";
  node [style=filled,
        shape="oval", 
        fixedsize=true, 
        width=1,
        height=1,
        fontsize=8
       ];
  edge [arrowhead=vee,
        arrowsize=0.5]
  
  {nodes}
  
  {edges}
  
  //{{rank = same; data; basicRobot; hardware; systemReef; SNAawareness}};
  
}}""".format(edges="\n  ".join(edges), 
             nodes="\n  ".join(node_descriptions)) # , babies=babies


# print(g)
g_lines = g.split("\n ")

digraph_js = """var digraph_details = {lines};

 digraph_details =  digraph_details.join("\\n");
  
""".format(lines=g_lines)

with open('digraph.js', 'w') as f:
    f.write(digraph_js)
    
jsonp_body += "\n\n\n" + digraph_js

In [40]:
# make the dictionary have the project name as a key, instead of bing a list of independent dicts
simple_dict = project_data.to_dict(orient="records")
new_dict = {}
for row in simple_dict:
    new_dict[row["Name"]] = row

j = simplejson.dumps(new_dict, ignore_nan=True, sort_keys=True, indent=2)
j = "var parallelData = {json};".format(json=j)
with open('parallelData.js', 'w') as f:
    f.write(j)
    
jsonp_body += "\n\n\n" + j

In [41]:
def get_thumb(row):
    try:
        h = row["Headshot"]
        if type(h) is not float:
            # TODO: make this check for small as well
            x = list(h)
            return x[0]['thumbnails']["large"]["url"]
        else:
            return "http://getdrawings.com/images/casper-drawing-8.jpg"
    except Exception as e:
        print(e)
        return "http://getdrawings.com/images/casper-drawing-8.jpg"


In [42]:
people_data = get_table("People")
people_data["thumb_large"] = people_data.apply(get_thumb, axis=1)
people_data.sample(5)

,Headshot,Initials,Name,Notes,Techs,id,thumb_large
3,"[{'size': 159204, 'thumbnails': {'small': {'ur...",BaD,Barry D,NaN,NaN,recFKWrjwOUM5oWxI,https://dl.airtable.com/4NWLSnTFRwSfdFDBAt9A_l...
0,"[{'size': 186823, 'thumbnails': {'small': {'ur...",KM,Kevin M,NaN,[recnXgQGgzh4q3t9M],rec0dpxUhHfgHWU7r,https://dl.airtable.com/6ax7C3bmSUWbWYULMZsZ_l...
2,"[{'size': 349692, 'thumbnails': {'small': {'ur...",--,nobodyAssigned,This is who'll be assigned automagically if no...,NaN,recF9tgC6LJUmi5gJ,https://dl.airtable.com/k7bmR1FNTyeJBIzaiWcA_l...
4,NaN,RX,AI Richard,NaN,[recnjMKU5MFlbshGD],recFUgthTUYQnR7Mz,http://getdrawings.com/images/casper-drawing-8...
10,"[{'size': 4270, 'thumbnails': {'small': {'url'...",AR,Aiden,NaN,"[rechAwGrx1K8hAmTi, recXEw0Q2LFFIa2XB, rec6MNL...",recj2rmnndUmxB3Dt,https://dl.airtable.com/wNAkSR0RUlLTf8AMRYJA_l...


In [43]:
# make the dictionary have the project name as a key, instead of bing a list of independent dicts
simple_people_dict = people_data.to_dict(orient="records")
new_p_dict = {}
for row in simple_people_dict:
    new_p_dict[row["id"]] = row
    
new_p_dict

j = simplejson.dumps(new_p_dict, ignore_nan=True, sort_keys=True, indent=2)
j = "var peopleData = {json};".format(json=j)
with open('peopleData.js', 'w') as f:
    f.write(j)
jsonp_body += "\n\n\n" + j

In [44]:
with open('all.js', 'w') as f:
    f.write(jsonp_body)  

![](https://dl.airtable.com/vao3Ge65RXuQs17gLU4p_small_MVIMG_20180224_173642.jpg)